In [1]:
%load_ext autoreload

In [2]:
import sys
import os

coursework_path = '/home/divanik/CourseWork/Nystrom_TT_rounding_and_convolution'
sys.path.insert(0, coursework_path)

In [3]:
import tt
import numpy as np
import time

In [4]:
def create_inverse_index_sums_tensorized(n, d, QTT_d):
    nats = tt.vector.to_list(tt.xfun(n, QTT_d))
    base = tt.vector.to_list(tt.ones(n, QTT_d * d))
    X = []
    for i in range(d):
        tmp = base.copy()
        for j in range(QTT_d):
            tmp[i * QTT_d + j] = nats[j].copy()
        X.append(tt.vector.from_list(tmp))
    c = tt.multifuncrs2(X, lambda x: 1 / (1 + np.sum(x, axis = 1)), eps=1E-8, verb=0)
    return c

In [8]:
def count_time(timed_dict, streams):
    time1 = timed_dict['left_random'] + timed_dict['left_contraction']
    time2 = timed_dict['right_random'] + timed_dict['right_contraction']
    def take_div(div, mod, lst):
        answer = 0
        for i in range(mod, len(lst), div):
            answer += lst[i]
        return answer
    time1 += take_div(2, 0, timed_dict['psi']) + take_div(2, 0, timed_dict['phi']) + take_div(2, 0, timed_dict['invert'])
    time2 += take_div(2, 1, timed_dict['psi']) + take_div(2, 1, timed_dict['phi']) + take_div(2, 1, timed_dict['invert'])
    if streams == 1:
        return time1 + time2
    elif streams == 2:
        return max(time1, time2)
    else:
        raise "Emulation is not implemented"

In [5]:
hadamard_factor = tt.vector.to_list(create_inverse_index_sums_tensorized(2, 2, 30))

In [6]:
tt.vector.from_list(hadamard_factor)

This is a 60-dimensional tensor 
r(0)=1, n(0)=2 
r(1)=2, n(1)=2 
r(2)=4, n(2)=2 
r(3)=8, n(3)=2 
r(4)=16, n(4)=2 
r(5)=17, n(5)=2 
r(6)=19, n(6)=2 
r(7)=20, n(7)=2 
r(8)=22, n(8)=2 
r(9)=24, n(9)=2 
r(10)=25, n(10)=2 
r(11)=27, n(11)=2 
r(12)=29, n(12)=2 
r(13)=30, n(13)=2 
r(14)=32, n(14)=2 
r(15)=34, n(15)=2 
r(16)=34, n(16)=2 
r(17)=36, n(17)=2 
r(18)=38, n(18)=2 
r(19)=39, n(19)=2 
r(20)=40, n(20)=2 
r(21)=42, n(21)=2 
r(22)=43, n(22)=2 
r(23)=45, n(23)=2 
r(24)=46, n(24)=2 
r(25)=47, n(25)=2 
r(26)=47, n(26)=2 
r(27)=49, n(27)=2 
r(28)=49, n(28)=2 
r(29)=50, n(29)=2 
r(30)=48, n(30)=2 
r(31)=47, n(31)=2 
r(32)=46, n(32)=2 
r(33)=45, n(33)=2 
r(34)=43, n(34)=2 
r(35)=42, n(35)=2 
r(36)=41, n(36)=2 
r(37)=39, n(37)=2 
r(38)=37, n(38)=2 
r(39)=36, n(39)=2 
r(40)=34, n(40)=2 
r(41)=33, n(41)=2 
r(42)=31, n(42)=2 
r(43)=30, n(43)=2 
r(44)=28, n(44)=2 
r(45)=28, n(45)=2 
r(46)=26, n(46)=2 
r(47)=25, n(47)=2 
r(48)=24, n(48)=2 
r(49)=22, n(49)=2 
r(50)=21, n(50)=2 
r(51)=20, n(51)=2 
r(5

In [7]:
def pad_tensor_to_max_rank(tensor):
    padded_tensor = []
    max_rank = 0
    for kernel in tensor:
        max_rank = max(max_rank, kernel.shape[0])
        max_rank = max(max_rank, kernel.shape[2])
    for idx, kernel in enumerate(tensor):
        if idx == 0:
            kernel1 = np.zeros((1, kernel.shape[1], max_rank))
        elif idx == len(tensor) - 1:
            kernel1 = np.zeros((max_rank, kernel.shape[1], 1))
        else:
            kernel1 = np.zeros((max_rank, kernel.shape[1], max_rank))
        kernel1[:kernel.shape[0],:,:kernel.shape[2]] = kernel
        padded_tensor.append(kernel1)
    return padded_tensor

In [8]:
%autoreload 2
from algorithms_package.src import hadamard_product, convolution

In [9]:
def count_precision(precise_hadamard_product, supposed_rounding):
    return tt.vector.norm(precise_hadamard_product - supposed_rounding)

In [10]:
padded_tensor = pad_tensor_to_max_rank(hadamard_factor)
ttpy_tensor = tt.vector.from_list(padded_tensor)

In [11]:
reference_hadamard_product = tt.vector.from_list(hadamard_factor) * tt.vector.from_list(hadamard_factor)

In [12]:
precise_hadamard_product = tt.vector.from_list(padded_tensor) * tt.vector.from_list(padded_tensor)

In [13]:
well_approx = reference_hadamard_product.round(0, 100)

In [18]:
kek = precise_hadamard_product.round(0, 50)

In [19]:
tt.vector.norm(kek - precise_hadamard_product)

1.4926703747405583e-13

In [50]:
d = len(hadamard_factor)
df = []

for rounding_rank in range(50, 51, 3):
    for retry in range(5):
        times = []
        precisions = []
        names = []
        # timestamp0 = time.time()
        # answer1 = precise_hadamard_product.round(0, rounding_rank)
        # timestamp1 = time.time()
        # answer2 = tt.multifuncrs2([ttpy_tensor, ttpy_tensor], lambda x : x[:, 0] * x[:, 1], eps=1e-6, verb = 0)
        timestamp2 = time.time()
        answer3 = hadamard_product.approximateHadamardProduct(padded_tensor, padded_tensor, [rounding_rank] * (d - 1), seed = rounding_rank * 4000000 + retry)
        timestamp3 = time.time()
        answer3 = tt.vector.from_list(answer3)
        # times.append(timestamp1 - timestamp0)
        # times.append(timestamp2 - timestamp1)
        times.append(timestamp3 - timestamp2)
        # precisions.append(count_precision(reference_hadamard_product, answer1))
        # precisions.append(count_precision(reference_hadamard_product, answer2))
        precisions.append(count_precision(reference_hadamard_product, answer3))
        # names.append('TT-rounding')
        # names.append('multifuncrs')
        names.append('RTOH')
        for kappa in [1.0, 1.5]:
            auxiliary_rank = max(int(kappa * rounding_rank), rounding_rank) 
            answer4, timing = hadamard_product.approximateTwoSidedHadamardProduct(padded_tensor, padded_tensor, [rounding_rank] * (d - 1), [auxiliary_rank] * (d - 1), seed = 4000000 * rounding_rank + 2000 * auxiliary_rank + retry)
            times.append(count_time(timing, 1))
            times.append(count_time(timing, 2))
            precision = count_precision(reference_hadamard_product, tt.vector.from_list(answer4))
            precisions.append(precision)
            precisions.append(precision)
            names.append(f'TSRH(kappa={kappa}, 1thread)')
            names.append(f'TSRH(kappa={kappa}, 2thread)')
        for time_, precision, name in zip(times, precisions, names):
            df{'time_' : time_, 'precision' : precision, 'rounding_rank' : rounding_rank, 'name' : name}



In [29]:
d = len(hadamard_factor)
df = []

for rounding_rank in range(35, 81, 5):
    print('Rounding rank: ', rounding_rank)
    for retry in range(5):
        print('Retry: ', retry)
        for kappa in [1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0]:
            print('Kappa: ', kappa)
            auxiliary_rank = max(int(kappa * rounding_rank), rounding_rank) 
            answer4, timing = hadamard_product.approximateTwoSidedHadamardProduct(padded_tensor, padded_tensor, [rounding_rank] * (d - 1), [auxiliary_rank] * (d - 1), seed = 4000000 * rounding_rank + 2000 * auxiliary_rank + retry)
            precision = count_precision(well_approx, tt.vector.from_list(answer4))
            df.append({'precision' : precision, 'rounding_rank' : rounding_rank, 'kappa' : kappa})
            print(precision)


Rounding rank:  35
Retry:  0
Kappa:  1.0
1.0667581099868625e-06
Kappa:  1.5
4.865275302894857e-07
Kappa:  2.0
4.282044662665987e-07
Kappa:  2.5
2.213401716484521e-07
Kappa:  3.0
3.213736133069505e-07
Kappa:  3.5
1.3344122956364055e-07
Kappa:  4.0
5.168109556481061e-08
Retry:  1
Kappa:  1.0
5.512810557228747e-07
Kappa:  1.5
7.599364709166024e-07
Kappa:  2.0
3.1331979115841796e-07
Kappa:  2.5
1.9114841932066939e-07
Kappa:  3.0
1.1467289145044758e-07
Kappa:  3.5
3.2937196052748035e-07
Kappa:  4.0
1.8166910578152632e-07
Retry:  2
Kappa:  1.0
2.916885086348547e-06
Kappa:  1.5
3.9467020141938477e-07
Kappa:  2.0
7.970938497233799e-07
Kappa:  2.5
2.7788587794774033e-07
Kappa:  3.0
8.594913307734125e-08
Kappa:  3.5
1.125152321036219e-07
Kappa:  4.0
1.419939269046352e-07
Retry:  3
Kappa:  1.0
2.3893272397366035e-06
Kappa:  1.5
4.945695968810971e-07
Kappa:  2.0
4.4740701846210215e-07
Kappa:  2.5
5.205554917447435e-07
Kappa:  3.0
1.6956878224886893e-07
Kappa:  3.5
1.3980086148556581e-07
Kappa:  4.

In [20]:
import pandas as pd

In [21]:
d = len(hadamard_factor)
df = []

for rounding_rank in range(75, 81, 5):
    print('Rounding rank: ', rounding_rank)
    for retry in range(5):
        print('Retry: ', retry)
        for oversampling in [0, 1, 2, 3, 5, 7, 10]:
            print('Oversampling: ', oversampling)
            auxiliary_rank = 3 * (rounding_rank + oversampling)
            answer4, timing = hadamard_product.approximateTwoSidedHadamardProduct(padded_tensor, padded_tensor, [rounding_rank + oversampling] * (d - 1), [auxiliary_rank] * (d - 1), seed = 4000000 * rounding_rank + 2000 * auxiliary_rank + retry)
            answer4 =  tt.vector.from_list(answer4).round(0, rounding_rank)
            precision = count_precision(well_approx, answer4)
            print(precision)
            df.append({'precision' : precision, 'rounding_rank' : rounding_rank, 'oversampling' : oversampling})

Rounding rank:  75
75
Retry:  0
Oversampling:  0
1.704202471576592e-10
Oversampling:  1
8.371159725586587e-11
Oversampling:  2
1.7333295943609478e-10
Oversampling:  3
2.948145966686832e-11
Oversampling:  5
7.637313636770546e-11
Oversampling:  7
3.529673001927078e-11
Oversampling:  10
3.368227297855782e-11
Retry:  1
Oversampling:  0
1.0009658096099514e-10
Oversampling:  1
8.318347734477715e-11
Oversampling:  2
6.656012475812134e-11
Oversampling:  3
4.5934818455762086e-11
Oversampling:  5
8.699090157418631e-11
Oversampling:  7
5.0416757886871756e-11
Oversampling:  10
3.080214047805695e-11
Retry:  2
Oversampling:  0
5.254866347803382e-11
Oversampling:  1
6.435074977762249e-11
Oversampling:  2
1.2592710784584149e-10
Oversampling:  3
9.201800585792897e-11
Oversampling:  5
3.521038907064176e-11
Oversampling:  7
7.288893733428258e-11
Oversampling:  10
4.049553146220011e-11
Retry:  3
Oversampling:  0
8.316132387290053e-11
Oversampling:  1
1.1636320876277631e-10
Oversampling:  2
2.2974760718423

In [2]:
import pandas as pd

In [3]:
pd.DataFrame(df).to_csv(f'{coursework_path}/data/experiment3/oversampling_dependence.csv')

NameError: name 'df' is not defined

In [156]:
for rank in range(10, 16, 5):
    for oversampling in range(4, 8, 3):
        wow = 10000 * rank + 100 * oversampling
        for seed in range(wow, wow + 7, 2):
            time_begin = time.time()
            approximate_product = tt.vector.from_list(hadamard_product.approximateTwoSidedHadamardProduct(hadamard_factor, hadamard_factor, [rank] * 59, [rank + oversampling] * 59, seed, leave_left = True))
            result_time = time.time() - time_begin 
            result_precision = tt.vector.norm(approximate_product - precise_product) / big_norm
            print(rank, oversampling, seed)
            answer.append({'rounding_rank' : rank, 'oversampling_rank' : rank + oversampling, 'seed' : seed, 'precision': result_precision, 'time': result_time})

/home/divanik/anaconda3/lib/python3.9/site-packages/tt/core/vector.py:143: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if d is 2:
/home/divanik/anaconda3/lib/python3.9/site-packages/tt/core/vector.py:495: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if dt.size is 1:
/home/divanik/anaconda3/lib/python3.9/site-packages/tt/core/vector.py:143: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if d is 2:
/home/divanik/anaconda3/lib/python3.9/site-packages/tt/core/vector.py:495: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if dt.size is 1:


AttributeError: 'list' object has no attribute 'flatten'

In [15]:
import pandas as pd

In [17]:
kek = pd.DataFrame(answer)

In [66]:
for rank in range(10, 46, 5):
    for oversampling in range(-5, 21, 3):
        wow = 10000 * rank + 100 * oversampling
        for seed in range(wow, wow + 7, 2):
            time_begin = time.time()
            approximate_product = tt.vector.from_list(hadamard_product.approximateTwoSidedHadamardProduct(hadamard_factor, hadamard_factor, [rank] * 59, [rank + oversampling] * 59, seed, leave_left = True))
            result_time = time.time() - time_begin 
            result_precision = tt.vector.norm(approximate_product - precise_product) / big_norm
            print(rank, oversampling, seed)
            answer.append({'rounding_rank' : rank, 'oversampling_rank' : rank_oversampling, 'seed' : seed, 'precision': result_precision, 'time': result_time})

[array([[[ 1.05559872e-57, -4.57008710e-58,  1.50236631e-57,
          4.31396593e-59,  3.65049633e-57, -1.45914549e-57,
          3.47920386e-58, -2.00895473e-57, -1.80940925e-57,
         -1.88812454e-58, -2.68531828e-57,  9.57431360e-58,
          2.47178332e-57, -3.05731455e-57,  6.36015972e-58,
          9.91585248e-58,  5.22629370e-58, -8.66995294e-58,
         -9.05264413e-58,  1.51242606e-57, -2.43455584e-57,
         -1.89778763e-57, -3.13481140e-57, -2.98425999e-58,
         -3.16621492e-58, -1.42490477e-57, -2.05208486e-57,
         -2.25380015e-57,  6.37223347e-58, -2.16591016e-58,
          2.68139671e-57, -1.62904752e-57,  1.68385656e-57,
          1.68071413e-57,  2.42261341e-58,  2.12005416e-57,
         -1.60842308e-57,  1.56340411e-58,  2.53509218e-57,
          2.36576762e-58],
        [ 3.16317708e-58, -2.94987951e-58,  3.74268750e-58,
         -1.58606909e-58,  1.14159098e-57, -5.28478736e-58,
          6.13466934e-59, -6.77366343e-58, -4.88358484e-58,
         -1.

In [46]:
big_norm = tt.vector.norm(precise_product)

In [67]:
tt.vector.norm(approximate_product - precise_product) / big_norm

5.241649544867888e-05

In [ ]:
answer = []

In [17]:
d = len(hadamard_factor)
df = []

for rounding_rank in range(40, 81, 5):
    for retry in range(5):
        for oversampling in [0, 1, 2, 3, 5, 7]:
            answer4 = hadamard_product.approximateHadamardProduct(padded_tensor, padded_tensor, [rounding_rank + oversampling] * (d - 1), seed = 4000000 * rounding_rank + 2000 * oversampling + retry)
            answer4 =  tt.vector.from_list(answer4).round(0, rounding_rank)
            precision = count_precision(well_approx, answer4)
            print(precision, rounding_rank, oversampling)
            df.append({'precision' : precision, 'rounding_rank' : rounding_rank, 'oversampling' : oversampling})

2.523350571571782e-10 40 0
1.965769961690531e-10 40 1
6.171813409516344e-11 40 2
4.5009328316978145e-11 40 3
2.2528855203758896e-11 40 5
1.4491961956062953e-11 40 7
2.0607918285175634e-10 40 0
1.4219200009440612e-10 40 1
8.753696406964404e-11 40 2
3.320612323019735e-11 40 3
2.0243548659552228e-11 40 5
1.075206336813563e-11 40 7
1.406546541231357e-10 40 0
1.8409466864817965e-10 40 1
1.3755565635173988e-10 40 2
4.9724515311320454e-11 40 3
5.068082793780497e-11 40 5
1.1968887109782819e-11 40 7
2.2807442363784822e-10 40 0
1.3063597747142951e-10 40 1
7.881686893837502e-11 40 2
9.860436616700884e-11 40 3
2.5830552988735915e-11 40 5
1.3478002915312524e-11 40 7
3.970184758828119e-10 40 0
1.4911976503547735e-10 40 1
6.483784445145207e-11 40 2
6.100593927141991e-11 40 3
1.825619884392227e-11 40 5
1.346726144222235e-11 40 7
1.7544565695610628e-11 45 0
9.605875553943886e-12 45 1
8.25344591271211e-12 45 2
9.743406158197213e-12 45 3
1.4410143954073222e-12 45 5
1.8676540717044417e-12 45 7
4.509965110

In [19]:
import pandas as pd

In [20]:
pd.DataFrame(df).to_csv(f'{coursework_path}/data/experiment2/RTO_oversampling_dependence.csv')